# Context-augmented generation (user-level context)

This notebook demonstrates a **context-augmented generation** pattern using a LLM. Instead of performing a retrieval step for every user query, we identify the relevant domain information up front and inject it directly into the user prompt.

In this example, the context is injected into the **user prompt**.


## 1. Setup

This section configures the OpenAI client.

> **Note**: To run the notebook, you need an `OPENAI_API_KEY` environment variable. If this env is not defined, you will be asked for your OpenAI API key (it will not be stored in the notebook).

In [1]:
import os
from getpass import getpass
from textwrap import dedent
from openai import OpenAI

# Ask for the API key interactively if not defined as env variable
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

client = OpenAI()

Enter your OpenAI API key: ··········


## 2. Function to request model

This section defines a helper function to send a user prompt to the model using the previously defined OpenAI client.

In [6]:
def send_prompt_to_model(prompt: str,
                  model: str = "gpt-4.1-mini",
                  max_output_tokens: int = 400) -> str:
    """Send a plain text prompt to an OpenAI model using the Responses API."""
    response = client.responses.create(
        model=model,
        input=prompt,
        max_output_tokens=max_output_tokens,
    )
    # The output format may evolve over time; this is correct at the time of writing.
    return response.output[0].content[0].text

## 3. Define external knowledge

To keep the example focused, we use a compact, fictional domain: a maintenance and troubleshooting guide for a professional espresso machine. In a real application, this could be a long technical manual, a set of
compliance rules, or a curated internal knowledge base.

In a **retrieval-augmented generation (RAG)** pipeline, this manual might be chunked into small fragments, embedded, and searched on every query. In **context-augmented generation**, we instead load the entire manual
into the model's context windows at the beginning of the interaction.

In [3]:
manual_text = """# Contoso Barista Pro 3000 – Maintenance & Troubleshooting Guide

This document describes key procedures for safely operating, cleaning, and troubleshooting
the Contoso Barista Pro 3000 espresso machine.

## 1. Daily operation

- Allow the machine to warm up for at least 15 minutes before brewing.
- Always use filtered water to reduce scale buildup.
- Do not run the pump without water in the reservoir.

## 2. Descaling procedure

1. Turn off the steam function and let the machine cool down for at least 20 minutes.
2. Empty the drip tray and remove the portafilter.
3. Fill the water reservoir with a descaling solution according to the manufacturer’s instructions.
4. Run a cleaning cycle until the reservoir is half empty.
5. Let the machine rest for 10 minutes so the descaling solution can dissolve mineral deposits.
6. Finish the cleaning cycle and then rinse the reservoir thoroughly with fresh water.
7. Run two full tanks of fresh water through the system before brewing coffee again.

## 3. Steam wand cleaning

- After every use, wipe the steam wand with a damp cloth.
- Purge steam for 3–5 seconds to clear residual milk.
- Once per week, soak the tip of the steam wand in warm water with a food-safe detergent.

## 4. Error codes

- **E10**: Water reservoir is empty or not seated correctly.
- **E12**: Pump is overheated. Turn off the machine and wait 30 minutes before restarting.
- **E17**: Descale cycle incomplete. The machine detected residual descaling solution or scale.
- **E32**: Temperature sensor fault. Contact a qualified service technician.

## 5. Resolving error code E17

1. Confirm that the descaling procedure from Section 2 was completed, including both rinse cycles.
2. Inspect the water reservoir and ensure it is filled with fresh water only.
3. Run one additional rinse cycle with fresh water.
4. If the error persists, unplug the machine for 5 minutes and restart it.
5. If E17 still appears, contact customer support with the machine’s serial number.

## 6. Safety notes

- Never open the machine housing while it is connected to power.
- Hot surfaces and pressurized steam can cause burns.
- Always allow the machine to cool completely before performing internal maintenance.
"""

print(manual_text[:800] + "\n...")

# Contoso Barista Pro 3000 – Maintenance & Troubleshooting Guide

This document describes key procedures for safely operating, cleaning, and troubleshooting
the Contoso Barista Pro 3000 espresso machine.

## 1. Daily operation

- Allow the machine to warm up for at least 15 minutes before brewing.
- Always use filtered water to reduce scale buildup.
- Do not run the pump without water in the reservoir.

## 2. Descaling procedure

1. Turn off the steam function and let the machine cool down for at least 20 minutes.
2. Empty the drip tray and remove the portafilter.
3. Fill the water reservoir with a descaling solution according to the manufacturer’s instructions.
4. Run a cleaning cycle until the reservoir is half empty.
5. Let the machine rest for 10 minutes so the descaling solution can d
...


## 4. Building a context-augmented prompt

The key idea of context-augmented generation is to inject the **complete domain reference** directly into the prompt, instead of retrieving only a few chunks per query.

We combine three elements into a single prompt string:

1. High-level system-style instructions that describe the assistant's role and behavior.
2. The full domain reference document (here, the coffee machine manual).
3. The user's question.

The model receives all three pieces at once, so it can reason over the entire manual, including cross-references
between sections.

In [4]:
base_instructions = """You are a support assistant for the Contoso Barista Pro 3000 espresso machine.

Use ONLY the information in the reference manual provided below to answer questions.
If the manual does not contain the answer, say that the information is not available.
Explain your reasoning in clear, step-by-step language that a non-expert user can follow.
"""

def build_context_augmented_prompt(user_question: str) -> str:
    """Construct a single text prompt that includes instructions, the full manual,
    and the user question.
    """
    prompt = f"""{base_instructions}

====================
REFERENCE MANUAL
====================
{manual_text}

====================
USER QUESTION
====================
{user_question}

When answering, cite the specific sections or steps from the manual that you are using.
"""
    return prompt


example_question = "The machine shows error code E17 after I finished descaling. What should I do?"

print(build_context_augmented_prompt(example_question)[:800] + "\n...")


You are a support assistant for the Contoso Barista Pro 3000 espresso machine.

Use ONLY the information in the reference manual provided below to answer questions.
If the manual does not contain the answer, say that the information is not available.
Explain your reasoning in clear, step-by-step language that a non-expert user can follow.


REFERENCE MANUAL
# Contoso Barista Pro 3000 – Maintenance & Troubleshooting Guide

This document describes key procedures for safely operating, cleaning, and troubleshooting
the Contoso Barista Pro 3000 espresso machine.

## 1. Daily operation

- Allow the machine to warm up for at least 15 minutes before brewing.
- Always use filtered water to reduce scale buildup.
- Do not run the pump without water in the res
...


## 5. Asking the model with context-augmented generation

Now we can send the context-augmented prompt to the model. Because the full manual is available in the prompt, the model can synthesize information from different sections (for example, the descaling procedure and the specific guidance for error code E17).


In [7]:
response = send_prompt_to_model(build_context_augmented_prompt(example_question))
print(response)

The error code E17 means the descale cycle was incomplete and the machine detected residual descaling solution or scale (see Section 4: Error codes).

To resolve E17, follow these steps from Section 5: Resolving error code E17:

1. Confirm you completed the full descaling procedure in Section 2, including running two full tanks of fresh water through the system after descaling (the rinse cycles).
2. Check that the water reservoir is filled only with fresh water.
3. Run one additional rinse cycle with fresh water.
4. If E17 still appears, unplug the machine for 5 minutes, then restart it.
5. If the error remains, contact customer support with your machine’s serial number.

Following these steps ensures all residual descaling solution or scale is flushed out, which is why the error is showing. The key is carefully performing the full descaling and rinse cycles as detailed in Section 2, then additional rinsing and restart as outlined in Section 5.


You can try additional questions that require the model to navigate the manual as a whole, for example:

- Questions that refer indirectly to content ("The pump feels hot and the machine stopped mid-shot. What error could that be?").
- Questions that require combining multiple sections ("How can I reduce scale buildup and avoid error E17 in the future?").

Because the entire manual is always present in the context, the model does not depend on a separate retrieval step to locate the relevant information.


## 6. Adapting this notebook to your own domain

To reuse this notebook in your own projects:

1. Replace the `manual_text` variable with the text of your domain reference (for example, a policy document, a product manual, or an internal knowledge base).
2. Adjust `base_instructions` to reflect the assistant's role, tone, and allowed sources of truth.
3. Tune the model and token limits as needed for your context window and latency constraints.

This notebook is intentionally self-contained so that you can experiment with context-augmented generation without requiring any external infrastructure beyond an LLM API.